### **Imports**
***

In [33]:
import pandas as pd
import json
import ast
import re
import os
import math
import numpy as np
import sys
import numpy as np
import pandas as pd
import warnings

from typing import List
from scipy.spatial.distance import cdist
from gensim.models import FastText, Word2Vec
from typing import List
from collections import Counter
from glob import glob
from difflib import SequenceMatcher
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

### **Read Player Feature Data**
---

In [ ]:
ground_truth_player_features_fp = '/playpen-storage/levlevi/player-re-id/src/data/raw_features.json'
ground_truth_player_features_df_fp = '/playpen-storage/levlevi/player-re-id/src/data/team_rosters_df.csv'
with open(ground_truth_player_features_fp, 'r') as f:
    ground_truth_player_features_dict = json.load(f)
ground_truth_player_features_df = pd.read_csv(ground_truth_player_features_df_fp)

In [ ]:
# which features are redundant?
ground_truth_player_features_subset_df = ground_truth_player_features_df[['team_id', 'player_id', 'jersey_number']]

# add feature to indicate if player is in game
ground_truth_player_features_subset_df['player_in_game'] = [False] * len(ground_truth_player_features_subset_df)

### **Format Tracklet Predicted Features**
***

In [ ]:
# human labels for 50 tracklet test set
with open('/playpen-storage/levlevi/player-re-id/src/data/_50_game_reid_benchmark_/annotations.json') as f:
    tracklet_human_annotations_dict = json.load(f)
    
# convert raw florence preditions to dict
florence_predictions_fp = '/playpen-storage/levlevi/player-re-id/src/data/florence_100_track_bm_results.json'
def read_florence_predictions(fp):
    with open(fp, 'r') as f:
        florence_predictions = f.readlines()
    predictions = {}
    for p in florence_predictions:
        key = p.split(':')[0]
        values = ast.literal_eval('{' + '{'.join(p.split('{')[1: ]))
        predictions[key] = values
    return predictions
predicted_tracklet_features_florence_dict = read_florence_predictions(florence_predictions_fp)

# roster metadata
with open("/playpen-storage/levlevi/player-re-id/src/data/raw_features.json", 'r') as f:
    ground_truth_player_features_dict = json.load(f)

In [ ]:
# get most common race from list
def get_maj_race(races):
    if len(races) == 0:
        return None
    counter = Counter(races)
    return counter.most_common(1)[0][0]

# get most common player postion from list
def get_maj_position(positions):
    if len(positions) == 0:
        return None
    counter = Counter(positions)
    return counter.most_common(1)[0][0]

# get most common jersey number from list
def get_maj_jersey_number(jersey_numbers):
    if len(jersey_numbers) == 0:
        return None
    counter = Counter(jersey_numbers)
    return counter.most_common(1)[0][0]

In [ ]:
HUDL_GAME_LOGS_DIR = '/mnt/sun/levlevi/nba-plus-statvu-dataset/hudl-game-logs'
# id: fp
game_logs_map = {
    f.split('.')[-2]: os.path.join(HUDL_GAME_LOGS_DIR, f) for f in os.listdir(HUDL_GAME_LOGS_DIR)
}

def calculate_string_similarity_score(string_one, string_two):
    return SequenceMatcher(None, string_one, string_two).ratio()

def find_closest_player_id(player_name: str) -> int:
    best_similarity = -float('inf')
    best_player_id_match = None
    for _, team_data in ground_truth_player_features_dict.items():
        for player, player_data in team_data['players'].items():
            # convert player name to full name
            full_name = player if ',' not in player else ' '.join(reversed(player.split(',')))
            player_id = player_data['player_id']
            similarity = calculate_string_similarity_score(full_name, player_name)
            if similarity > best_similarity:
                best_similarity = similarity
                best_player_id_match = player_id
    if best_player_id_match is None:
        raise ValueError(f"No matching player found for the name: {player_name}")
    return int(best_player_id_match)

def get_candidate_player_ids(tracklet_fp: str) -> List[str]:
    game_id = tracklet_fp.split('/')[-2].split('_')[0]
    period = tracklet_fp.split('/')[-2].split('_')[-1].split('period')[1][0]
    period = int(period)
    df = pd.read_csv(game_logs_map[game_id], delimiter=';')
    df['is_period'] = df['half'].apply(lambda x: x == period)
    df_period_matched = df[df['is_period'] == True]
    
    unique_player_ids = set()
    unique_player_names = set(df['player_name'].unique())
    unique_player_names = set(df_period_matched['player_name'].unique())
    unique_player_names_no_nan = {p for p in unique_player_names if not pd.isna(p)}
    for pn in unique_player_names_no_nan:
        unique_player_ids.add(find_closest_player_id(pn))
    return unique_player_ids

def get_candidate_team_ids(tracklet_fp: str) -> List[str]:
    team_one_name = tracklet_fp.split('/')[-2].split('_')[3].replace(" ", "_")
    team_two_name = tracklet_fp.split('/')[-2].split('_')[5].replace(" ", "_")
    team_one_id = ground_truth_player_features_dict[team_one_name]['team_id']
    team_two_id = ground_truth_player_features_dict[team_two_name]['team_id']
    return [team_one_id, team_two_id]

# all rows in predictions df
rows = []
for tracklet_fp, raw_predictions in predicted_tracklet_features_florence_dict.items():
    # get team ids from file path
    predicted_jersey_numbers = []
    candidate_team_ids = get_candidate_team_ids(tracklet_fp)
    candidate_players_ids = get_candidate_player_ids(tracklet_fp)
    # get jersey numbers
    for frame_idx in raw_predictions:
        # match all valid predictions
        temp_jersey_number_arr = re.findall(r'\d+', raw_predictions[frame_idx].get('<OCR>'))
        for n in temp_jersey_number_arr:
            predicted_jersey_numbers.append(n)
    # find most common jersey number
    maj_jersey_number = get_maj_jersey_number(predicted_jersey_numbers)
    temp_row = [tracklet_fp, maj_jersey_number, candidate_team_ids, candidate_players_ids]
    rows.append(temp_row)
    
# add all rows to df
all_tracklet_features = pd.DataFrame(rows, columns=['tracklet_file_path', 'predicted_jersey_number', 'candidate_team_ids', 'candidate_player_ids'])
all_tracklet_features.head()

In [ ]:
human_labels = []
for file_path in all_tracklet_features['tracklet_file_path']:
    video_name = file_path.split('/')[-2]
    subtrack = file_path.split('/')[-1]
    human_label = tracklet_human_annotations_dict[video_name]['tracks'][subtrack]['human_annotation']
    human_labels.append(human_label)
    
all_tracklet_features['human_label'] = human_labels

In [ ]:
JERSEY_VECTOR_SIZE = 500

# 1. generate ground truth player embeddings
jersey_numbers = list(set(ground_truth_player_features_subset_df['jersey_number'].astype(str)))
sentences = [[char for char in number] for number in jersey_numbers]
model = FastText(sentences, vector_size=JERSEY_VECTOR_SIZE, window=1, min_count=1, sg=1)

In [ ]:
unique_team_ids = list(ground_truth_player_features_dict[k]['team_id'] for k in ground_truth_player_features_dict)
unique_team_ids_idx_map = {str(int(team_id)): idx for idx, team_id in enumerate(unique_team_ids)}

unique_player_ids_arr = sorted(ground_truth_player_features_subset_df['player_id'].unique())
unique_player_ids_idx_map = {player_id: idx for idx, player_id in enumerate(unique_player_ids_arr)}

In [ ]:
def get_jersey_number_embed(number: str):
    if not number:
        return [-sys.maxsize + 1] * JERSEY_VECTOR_SIZE
    return model.wv[number]

def get_team_id_embed(team_ids: List[str]):
    embedding = np.zeros(len(unique_team_ids_idx_map))
    for team_id in team_ids:
        team_idx = unique_team_ids_idx_map[str(team_id)]
        embedding[team_idx] = 1
    return embedding
    
def get_player_id_embed(player_ids: List[str]):
    embedding = np.zeros(len(unique_player_ids_idx_map))
    for player_id in player_ids:
        player_idx = unique_player_ids_idx_map[player_id]
        embedding[player_idx] = 1
    return embedding

def normalize_embeddings(embeddings):
    return normalize(embeddings, norm='l2')

In [35]:
# flatten the player IDs and then one-hot encode
player_id_encoder = OneHotEncoder(categories=[unique_player_ids_arr], sparse=False)
player_id_encoder.fit(np.array(unique_player_ids_arr).reshape(-1, 1))  # fit the encoder

def get_player_id_embed_one_hot(player_ids: List[str], encoder):
    encoded = encoder.transform(np.array(list(player_ids)).reshape(-1, 1))
    return encoded.sum(axis=0)  # sum to get a single vector representing all candidate IDs
    
ground_truth_player_embeddings = []
for row in ground_truth_player_features_subset_df.itertuples():
    jersey_number = row.jersey_number
    # a. get jersey number embed
    jersey_number_embedding = get_jersey_number_embed(jersey_number)
    # b. get player id embeds
    player_ids = [row.player_id]
    player_id_embedding = get_player_id_embed_one_hot(player_ids, player_id_encoder)
    # c. get concatinated embedding
    concatinated_embedding = np.hstack([jersey_number_embedding, player_id_embedding])
    ground_truth_player_embeddings.append(concatinated_embedding)

ground_truth_player_embeddings = normalize_embeddings(ground_truth_player_embeddings)
ground_truth_player_features_subset_df['embedding'] = [np.ravel(embedding) for embedding in ground_truth_player_embeddings]
ground_truth_player_features_subset_df.head()

,team_id,player_id,jersey_number,player_in_game,embedding
0,1610612738,202340,0,False,"[-0.044721359549995794, -0.044721359549995794,..."
1,1610612738,203109,99,False,"[0.0005500481493427992, -0.0004607620234395495..."
2,1610612738,203089,30,False,"[-0.0004180342570888874, -0.000663657286698243..."
3,1610612738,1626154,28,False,"[-0.0005987632939214192, -0.000336613803565496..."
4,1610612738,201973,8,False,"[-0.0018356590344703974, -0.000657910193779705..."


In [ ]:
# 2. generated tracklet embeddings
tracklet_embeddings = []
for row in all_tracklet_features.itertuples():
    jersey_number = row.predicted_jersey_number
    candidate_player_ids = row.candidate_player_ids
    # a. get jersey number embed
    jersey_number_embedding = get_jersey_number_embed(jersey_number)
    # b. get player id embeds
    player_ids_embedding = get_player_id_embed_one_hot(list(candidate_player_ids), player_id_encoder)
    # c. get concatinated embedding
    concatinated_embedding = np.concatenate([jersey_number_embedding, player_ids_embedding])
    tracklet_embeddings.append(concatinated_embedding)
    
tracklet_embeddings = normalize_embeddings(tracklet_embeddings)
all_tracklet_features['embedding'] = [np.ravel(embedding) for embedding in tracklet_embeddings]
all_tracklet_features.head()

In [ ]:
print(all_tracklet_features.shape)

In [ ]:
# euclidean Distance
def euclidean_distance(vec1: List[float], vec2: List[float]) -> float:
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(vec1, vec2)))

# cosine Similarity
def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
    dot_product = sum(x * y for x, y in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(x ** 2 for x in vec1))
    magnitude2 = math.sqrt(sum(y ** 2 for y in vec2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0  # Avoid division by zero
    return dot_product / (magnitude1 * magnitude2)

def pearson_correlation(vec1: List[float], vec2: List[float]) -> float:
    n = len(vec1)
    sum1 = sum(vec1)
    sum2 = sum(vec2)
    sum1_sq = sum(x ** 2 for x in vec1)
    sum2_sq = sum(y ** 2 for y in vec2)
    product_sum = sum(x * y for x, y in zip(vec1, vec2))
    
    numerator = product_sum - (sum1 * sum2 / n)
    denominator = math.sqrt((sum1_sq - sum1 ** 2 / n) * (sum2_sq - sum2 ** 2 / n))
    if denominator == 0:
        return 0.0  # Avoid division by zero
    return numerator / denominator

### **Match Tracklet Embeddings to Candidates**
***

In [ ]:
# Initialize the result list with NaNs
reidentified_tracklet_player_ids = [np.nan] * len(human_labels)

# Extract ground truth player embeddings and IDs
ground_truth_embeddings = np.vstack(ground_truth_player_features_subset_df['embedding'].values)
ground_truth_player_ids = ground_truth_player_features_subset_df['player_id'].values

# Iterate over tracklet features
for tracklet_index, tracklet_row in enumerate(all_tracklet_features.itertuples()):
    tracklet_embedding = np.array(tracklet_row.embedding).reshape(1, -1)
    
    # Compute distances to all ground truth embeddings
    distances = cdist(tracklet_embedding, ground_truth_embeddings, metric='euclidean')
    
    # Find the index of the best match
    best_match_idx = np.argmin(distances)
    best_match_player_id = ground_truth_player_ids[best_match_idx]
    
    # Assign the best match player ID
    reidentified_tracklet_player_ids[tracklet_index] = best_match_player_id

In [ ]:
# create a dataframe with all predictions
predictions_computed = pd.DataFrame({'tracklet_file_path': all_tracklet_features['tracklet_file_path'], 'human_label': human_labels, 'prediction': reidentified_tracklet_player_ids})
predictions_computed.head()

In [ ]:
# find all matching predictions
matched_mask = predictions_computed['human_label'] == predictions_computed['prediction']
predictions_computed_matched = predictions_computed[matched_mask]

In [ ]:
# convert all ids to floats
predictions_computed_no_na = predictions_computed_matched.dropna()
predictions_computed_no_na['human_label'] = predictions_computed_no_na['human_label'].astype(int)
predictions_computed_no_na['prediction'] = predictions_computed_no_na['prediction'].astype(int)

# find all matching predictions
matched_mask = abs(predictions_computed_no_na['human_label'] - predictions_computed_no_na['prediction']) < 0.5
predictions_no_na_matched = predictions_computed_no_na[matched_mask]

In [ ]:
len(predictions_no_na_matched)